#Librería Request

In [ ]:
!pip install requests

#Métodos del ChatBot

In [ ]:
#Importar librerias
import json
import requests
import random
import cv2 #Libreria para la detección de rostros. entre otras cosas
#import os

#Variables para el Token y la URL del chatbot
TOKEN = "############################"
URL = "https://api.telegram.org/bot" + TOKEN + "/"

#Deteccion de Emociones (Son los metodos por los cuales se puede identificar las emociones, el mas eficaz a mi parecer el FisherFaces)
#method = 'EigenFaces'
method = 'FisherFaces'
#method = 'LBPH'

if method == 'EigenFaces': emotion_recognizer = cv2.face.EigenFaceRecognizer_create()
if method == 'FisherFaces': emotion_recognizer = cv2.face.FisherFaceRecognizer_create() #Parte del codigo que sirve con el metodo para la deteccion de rostros
if method == 'LBPH': emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()

emotion_recognizer.read('/content/drive/My Drive/Modelos2/modelo'+method+'.xml') #Lectura del modelo pre-entrenado
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml') 

def update(offset):
    #Llamar al metodo getUpdates del bot, utilizando un offset
    respuesta = requests.get(URL + "getUpdates" + "?offset=" + str(offset) + "&timeout=" + str(100))
    
    #Decodificar la respuesta recibida a formato UTF8
    mensajes_js = respuesta.content.decode("utf8")
 
    #Convertir el string de JSON a un diccionario de Python
    mensajes_diccionario = json.loads(mensajes_js)
 
    #Devolver este diccionario
    return mensajes_diccionario

def updateFoto(id):
    #Llamar al metodo getFile del bot
    respuestaF = requests.get(URL + "getFile?file_id=" + str(id))
    
    #Decodificar la respuesta recibida a formato UTF8
    mensajesF_js = respuestaF.content.decode("utf8")
 
    #Convertir el string de JSON a un diccionario de Python
    mensajesF_diccionario = json.loads(mensajesF_js)
 
    #Descargar imagen
    file_path= mensajesF_diccionario["result"]["file_path"]
    imagen = requests.get("https://api.telegram.org/file/bot############################/"+str(file_path)).content
    with open("imagen.jpg",'wb') as handler:
      handler.write(imagen)

def info_mensaje(mensaje):
 
    #Comprobar el tipo de mensaje
    if "text" in mensaje["message"]:
        tipo = "texto"
    elif "sticker" in mensaje["message"]:
        tipo = "sticker"
    elif "animation" in mensaje["message"]:
        tipo = "animacion" #Nota: GIF son animaciones
    elif "photo" in mensaje["message"]:
        tipo = "foto"
        id_file= mensaje["message"]["photo"][0]["file_id"]
        mensajesF_diccionario= updateFoto(id_file)
    else:
        tipo = "otro" # el resto de tipos entra en la categoria "otro"
 
    #Recoger la info del mensaje (remitente, id del chat e id del mensaje)
    nombre = mensaje["message"]["from"]["first_name"]
    id_chat = mensaje["message"]["chat"]["id"]
    id_update = mensaje["update_id"]
    if "username" in mensaje["message"]["from"]:
      usuario= mensaje["message"]["from"]["username"]
      msg="Ir al chat: t.me/"+usuario
    else:
      usuario=nombre
      msg="---------------------------------------------\nNueva Alerta\n\nUsuario: "+usuario+"\n\nHa intentado realizar un proceso pero no cuenta con un ID de Usuario.\n\nSe notificó al usuario que asigne el ID y vuelva a intentar.\n---------------------------------------------"
      requests.get(URL + "sendMessage?text=" + msg + "&chat_id=-1001146542990")
     
    #Devolver toda la informacion
    return tipo, id_chat, nombre, id_update, usuario, msg
 
def leer_mensaje(mensaje):
 
    #Extraer el texto, nombre de la persona e id del último mensaje recibido
    texto = mensaje["message"]["text"]
 
    #Devolver las dos id, el nombre y el texto del mensaje
    return texto
 
def enviar_mensaje(idchat, texto):
    #Llamar el metodo sendMessage del bot, passando el texto y la id del chat
    requests.get(URL + "sendMessage?text=" + texto + "&chat_id=" + str(idchat))
 
def enviar_foto(idchat, texto, imagen):
    #Llamar el metodo sendMessage del bot, passando el texto y la id del chat
    requests.get(URL + "sendPhoto?chat_id=" + str(idchat) + "&caption="+ str(texto) + "&photo=" + str(imagen) )

def notificar(notificacion):
    #Llamar el metodo sendMessage del bot, passando el texto y la id del chat
    requests.get(URL + "sendMessage?chat_id=############################&text="+notificacion)

def ReconocerEmocion(idchat):
    image = cv2.imread('/content/imagen.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    auxFrame=gray.copy()
    faces = faceClassif.detectMultiScale(gray,1.3,5)
    if len(faces) == 0:
      texto="¡Error!\nNo se ha podido reconocer tu rostro, esto podría deberse a:\n1. Condiciones de poca luz.\n2. Baja calidad/resolución.\n3. Uso de algun accesorio que dificulta el reconocimiento facial."
      requests.get(URL + "sendMessage?chat_id="+ str(idchat) +"&text="+texto)
    elif len(faces) == 1:
      for (x,y,w,h) in faces:
          rostro = auxFrame[y:y+h,x:x+w]
          rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
          result = emotion_recognizer.predict(rostro)
          print(str(result))
          if method == 'EigenFaces': P=5700
          if method == 'FisherFaces': P=500
          if method == 'LBPH': P=60
          #if result[0] == 0 and result[1] < P:
          #  texto="Dudoso:\n¿Pensando mucho el día de hoy?"
          if result[0] == 0 and result[1] < P:
            texto="Enojo:\n¿Mucha ira? Respira y tranquilizate un poco."
          elif result[0] == 1 and result[1] < P:
            texto="Felicidad:\nDebe ser un buen día para andar con esa alegría. :D"
          elif result[0] == 2 and result[1] < P:
            texto="Sorpresa:\n¿Sorprendido? Debe ser por algo impactante!"
          elif result[0] == 3 and result[1] < P:
            texto="Tristeza:\n¿Por qué esa cara? Sonríe que en la vida no hay tiempo para estar triste."
          else:
            texto="¡Error!\nAlgunos factores pueden estar impidiendo reconocer tu emoción:\n1. Condiciones de poca luz.\n2. Baja calidad/resolución.\n3. Quizá no estas viendo de frente a la cámara o estas usando algun accesorio."
          requests.get(URL + "sendMessage?chat_id="+ str(idchat) +"&text="+texto)
          
    else:
      texto="¡Advertencia! La detección de emociones no funciona con multiples rostros. (Se detectaron "+str(len(faces))+" para ser exactos)"
      requests.get(URL + "sendMessage?chat_id="+ str(idchat) +"&text="+texto)

#Estructura/Funciones del Chatbot

In [ ]:
#Variable para almacenar la ID del ultimo mensaje procesado
ultima_id = 0
Memoria= {}
MCodigo= {}
texto=""
while(True):
    mensajes_diccionario = update(ultima_id)
    r1 = random.randint(0, 100)
    for i in mensajes_diccionario["result"]:
 
        #Guardar la informacion del mensaje
        tipo, idchat, nombre, id_update, usuario, msg = info_mensaje(i)
        #print(idchat)
        
        #Memoria del Chatbot 3.0
        Memoria.update(Memoria.fromkeys(['/menu','menu','menú','/menú'],'\n¡Vamos a empezar!\nEnviame una foto de tu rostro (o la de alguien más) y determinaré tu emoción :D'))
        Memoria.update(Memoria.fromkeys(['/start','start'],'¡Bienvenid@!\nSoy P.A.M Tu Asistente Virtual de Detección de Emociones.'+Memoria['menu']))
        Memoria.update(Memoria.fromkeys(['ayuda','/ayuda'],'\n¿Algún problema con la detección de emociones? Digita SOPORTE y una agente se pondra en contacto contigo lo antes posible.\n\n-Administración P.A.M.'))
        Memoria.update(Memoria.fromkeys(['/hola','hola','ola'],"Hola, "+nombre+"!"+Memoria['menu']))
        Memoria.update(Memoria.fromkeys(['/adios','adios','adiós'],"Hasta pronto!, "+nombre))
        Memoria.update(Memoria.fromkeys(['/soporte','soporte'],"Se ha generado el Ticket de Soporte\n(Ticket N° "+str(id_update)+")\n\n¡La ayuda está en camino!\nNuestro equipo se contactará con usted lo antes posible.\n\n-Administración P.A.M."))
        MCodigo.update(MCodigo.fromkeys(['/soporte','/comprar','comprar','soporte'],"https://qrcode.tec-it.com/API/QRCode?data=Proceso%20N°:%20"+str(id_update)+"&errorcorrection=H"))
        Memoria.update(Memoria.fromkeys(['gracias','entendido','muchas gracias','gracias por todo'],"Un placer haberte asistido, hasta otra."))
 
        #Generar una respuesta dependiendo del tipo de mensaje
        if tipo == "texto":
            texto = leer_mensaje(i)
            if Memoria.get(texto.lower())!=None:
              if texto.lower() in MCodigo:
                texto_respuesta=Memoria.get(texto.lower())
                imagen=MCodigo.get(texto.lower())
                if idchat!=############################ and "username" in i["message"]["from"]:
                  if texto.lower()=="soporte" or texto.lower()=="/soporte":
                    notificacion="---------------------------------------------\nNuevo Ticket\n\nUsuario: "+nombre+"\n("+msg+")\n\nHa abierto un Ticket de Soporte:\nTicket N°: "+ str(id_update) +"\n\nAtender en lo antes posible.\n---------------------------------------------"
                  notificar(notificacion)
              else:
                texto_respuesta=Memoria.get(texto.lower())
            else:
              texto_respuesta="Lo siento no conozco el termino:\n" + str(texto) + "\n\n¿Necesitas algo?\nDigita AYUDA para ser asistido."
              
        elif tipo == "sticker":
            texto_respuesta = "Su sticker es "+ str(r1) + "% Bonito"
        elif tipo == "animacion":
            texto_respuesta = "Me gusta este GIF!"
        elif tipo == "foto":
            texto_respuesta = "¿Buscas algo relacionado a la imagen?\nDigita AYUDA para ser asistido."
        elif tipo == "otro":
            texto_respuesta = "¿Necesitas algo?\nDigita AYUDA para ser asistido."
 
        #Si la ID del mensaje es mayor que el ultimo, se guarda la ID + 1
        if id_update > (ultima_id-1):
            ultima_id = id_update + 1
 
        #Enviar la respuesta
        if texto.lower() in MCodigo:
          if "username" in i["message"]["from"]:
            enviar_foto(idchat, texto_respuesta, imagen)
          else:
            enviar_mensaje(idchat, "¡Error al completar el proceso!\nSu cuenta no tiene asignado un ID de Usuario, por lo que le recomendamos asigne uno y vuelva realizar el proceso.\n\nEnlaces de Interes:\n\nWeb: https://moviltoday.com/elegir-nombre-de-usuario-telegram/\n\nMovil: https://www.xataka.com/basics/como-usar-telegram-numero-telefono\n\n-Administración P.A.M")
        elif tipo=="foto":
          ReconocerEmocion(idchat)
        else:
          enviar_mensaje(idchat, texto_respuesta)
        
                     
    #Vaciar
    mensajes_diccionario = []
    Memoria = {}
    MCodigo = {}
    texto=""

(1, 72.72549162159679)
(3, 83.19435050520839)
(3, 206.37390913299728)
